In [1]:
%pylab inline
from pyiem.network import Table as NetworkTable
import pandas as pd
from pandas.io.sql import read_sql
from pyiem.plot import MapPlot
import numpy as np
from tqdm import tqdm
from pyiem.meteorology import windchill
from pyiem.datatypes import temperature, speed
from pyiem.util import get_dbconn

Populating the interactive namespace from numpy and matplotlib


In [3]:
ASOS = get_dbconn('asos')

MESOSITE = get_dbconn('mesosite')
mcursor = MESOSITE.cursor()

mcursor.execute("""
    SELECT id, network, st_x(geom) as lon, st_y(geom) as lat from stations where country = 'US' and 
    network ~* 'ASOS' and state not in ('AK', 'HI') and
    archive_begin < '1973-01-01' and archive_end is null and length(id) = 3""")

rows = []
progress = tqdm(mcursor, total=mcursor.rowcount)
for row in progress:
    progress.set_description(row[0])
    # Figure out what our archive coverage is, 41 years would be perfect
    # https://leafo.net/guides/postgresql-calculating-percentile.html#calculating-all-percentiles
    df = read_sql("""
        with data as (
        SELECT date_trunc('hour', valid) as hr, max(feel)::int as val from alldata
        where station = %s and valid > '1973-01-01' and feel is not null and feel < 200
        and extract(month from valid) in (4, 5, 6, 7, 8, 9)
        and report_type = 2 GROUP by hr)
        select percentile_disc(0.995) within group (ORDER by val) from data
    """, ASOS, params=(row[0],) )

    # 12 coldest hours over 6 winter months is 99.8th percentile
    # val = np.percentile(df['val'].values, np.arange(98.5, 100.1, 0.1))
    rows.append(dict(sid=row[0], lat=row[3], lon=row[2],
                     val=df.iat[0, 0], network=row[1]))
    # print sid, len(w), val

df = pd.DataFrame(rows)
df.set_index('sid', inplace=True)
df['useme'] = True


CEY:  10%|█         | 69/690 [16:33<1:56:20, 11.24s/it]


HUM:  20%|█▉        | 137/690 [27:08<1:10:18,  7.63s/it]


MGM:  30%|██▉       | 205/690 [37:48<1:24:56, 10.51s/it]


CAO:  40%|███▉      | 274/690 [48:43<1:05:06,  9.39s/it]


EVV:  50%|████▉     | 343/690 [1:00:14<53:59,  9.33s/it]


...

LOL:  69%|██████▉   | 479/690 [1:21:37<37:13, 10.59s/it]


MEH:  79%|███████▉  | 547/690 [1:32:20<22:06,  9.28s/it]


BWG:  89%|████████▉ | 615/690 [1:43:14<18:44, 15.00s/it]


DRT:  99%|█████████▉| 683/690 [1:54:58<01:04,  9.19s/it]


CRP: 100%|██████████| 690/690 [1:56:28<00:00, 10.13s/it]


In [4]:
#df.to_csv('heat_percentiles.csv')
#df.at['3A1', 'useme'] = False
#df.at['87Q', 'useme'] = False
print(df)

           lat        lon  val  network  useme
sid                                           
MCN  32.692850 -83.649210  105  GA_ASOS   True
MCO  28.429390 -81.308990  103  FL_ASOS   True
BAK  39.261920 -85.896330   98  IN_ASOS   True
BTV  44.470000 -73.150000   93  VT_ASOS   True
CVG  39.043060 -84.671670   99  KY_ASOS   True
..         ...        ...  ...      ...    ...
EYW  24.556111 -81.759556  105  FL_ASOS   True
PIT  40.491470 -80.232860   94  PA_ASOS   True
SHV  32.447200 -93.824400  107  LA_ASOS   True
JAN  32.320000 -90.080000  106  MS_ASOS   True
CRP  27.773060 -97.512780  108  TX_ASOS   True

[690 rows x 5 columns]


In [6]:
df2 = df[df['useme'] == True]
for percentile in [99.5, ]:
    # percentile = 100. - (p * 0.1)
    values = df2['val']
    hours = 182. * 24. * ((100. - percentile) / 100.)
    m = MapPlot(sector='conus',
                title=("%.1fth Percentile Hottest Heat Index Temperature (April thru September)"
                       ) % (percentile,),
                subtitle=("based on longterm automated stations hourly METARs, "
                          "%.1f is ~%.0f hours out of six months"
                         ) % (percentile, hours))
    m.contourf(np.array(df2.lon), np.array(df2.lat), np.array(values), np.arange(80, 111, 5), units='F',
              cmap=plt.get_cmap('jet'))
    m.plot_values(df2.lon, df2.lat, values, fmt='%.0f', labelbuffer=15)
    m.postprocess(filename='hdx_%.0f.png' % (percentile * 10,))
    m.close()